In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install joblib
!pip install better-profanity
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken 2ViSQqmpq6cREZ5l5bdgieOkpvo_6QwTCCsxngKtjHaPnRCut

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=ab414245283b89097f86fa1301782fae95ca5e42052ebce8394ab7373a3755ec
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from flask import Flask, request, jsonify
from better_profanity import profanity
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
import requests
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
#from profanity_check import predict, predict_prob


In [ ]:
from pickle import TRUE
#step 1: Profanity Checker

# text = 'I will kill you'
# print(profanity.contains_profanity(text))

def detect_profanity(text):
  # Check if the text contains profanity
  contains_profanity = profanity.contains_profanity(text)

  # If there is profanity, count the number of profanity words
  if contains_profanity:
      profanity_words = [word for word in text.split() if profanity.contains_profanity(word)]
      num_profanity_words = len(profanity_words)
      return True
  else:
      num_profanity_words = 0
      return False

# Print the result
# print(f"Contains Profanity: {contains_profanity}")
# print(f"Number of Profanity Words: {num_profanity_words}")

detect_profanity("this is a clean sentence")



False

In [ ]:
# Step 2: Salience Detection Model
# Load the dataset and preprocess it


def Saliency_detector(input_text):
  # Load the dataset
  data = pd.read_csv("Saliency_train_set.csv")

  # Convert 'Saliency' column to binary (0 or 1)
  data['saliency'] = data['saliency'].apply(lambda x: 0 if pd.isna(x) else 1)
  data.to_csv('updated_data.csv', index=False)

  # Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(data['text'], data['saliency'], test_size=0.2, random_state=42)

  # Create TF-IDF vectors for the text data
  tfidf_vectorizer = TfidfVectorizer()
  #tfidf_vectorizer = TfidfVectorizer()
  X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
  X_test_tfidf = tfidf_vectorizer.transform(X_test)

  # Train a salience detection model (SVM for example)
  salience_model = SVC(kernel='linear')
  salience_model.fit(X_train_tfidf, y_train)
  #####################################################

  # Convert the input text into TF-IDF vector format
  input_text_tfidf = tfidf_vectorizer.transform([input_text])

  # Use the trained model to predict saliency
  prediction = salience_model.predict(input_text_tfidf)

  # Interpret the prediction
  if prediction[0] == 1:
      return 1
  else:
      return 0
Saliency_detector("how to reset my hard drive")


1

In [1]:
# Step 3: User Interface as a REST API using Flask

app = Flask(__name__)
run_with_ngrok(app)
@app.route('/detect_salient', methods=['POST'])
def detect_salient():


  input_text = request.json['text']
  salient_sentences = []
  non_salient_sentences = []
  profanity_sentences = []



  for sentences in input_text:
    salience_prediction=Saliency_detector(sentences)
      # if detect_profanity(sentence):
      #     continue  # Skip profane sentences

      # sentence_tfidf = tfidf_vectorizer.transform([sentence])
      # salience_prediction = salience_model.predict(sentence_tfidf)[0]
    if detect_profanity(sentences):
      non_salient_sentences.append(sentences)
      profanity_sentences.append(sentences)
    else:
      if salience_prediction == 1:
          salient_sentences.append(sentences)
      else:
          non_salient_sentences.append(sentences)

  response = {
      'salient_sentences': salient_sentences,
      'non_salient_sentences': non_salient_sentences,
      'profanity_sentences': profanity_sentences
  }




  return jsonify(response)

NameError: ignored

In [ ]:
def evaluate_and_save_predictions(data_file, output_file):
    # Load the data
    data = pd.read_csv(data_file)

    # Apply the Saliency_detector function to each cell in the "text" column

    ###########################################################
    data['prediction'] = data['text'].apply(Saliency_detector)
    #############################################################

    # Calculate evaluation metrics
    data['saliency'] = data['saliency'].apply(lambda x: 0 if pd.isna(x) else 1)
    y_true = data['saliency']
    y_pred = data['prediction']

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    confusion = confusion_matrix(y_true, y_pred)

    # Print the evaluation metrics
    print("Accuracy:", accuracy, " (Proportion of correctly classified instances)")
    print("Precision:", precision, " (Proportion of true positives among all predicted positives)")
    print("Recall:", recall, " (Proportion of true positives among all actual positives)")
    print("F1-Score:", f1, " (Harmonic mean of precision and recall, balances false positives and false negatives)")
    print("Confusion Matrix:")

    print("\t\tActual Positive\tActual Negative")
    print(f"Predicted Positive\t{confusion[0][0]}\t\t{confusion[0][1]}")
    print(f"Predicted Negative\t{confusion[1][0]}\t\t{confusion[1][1]}")



    # Save the DataFrame with predictions to a new CSV file
    data.to_csv(output_file, index=False)
evaluate_and_save_predictions("evaluation_data.csv", "prediction_data.csv")

Accuracy: 0.8604651162790697  (Proportion of correctly classified instances)
Precision: 0.8947368421052632  (Proportion of true positives among all predicted positives)
Recall: 0.8095238095238095  (Proportion of true positives among all actual positives)
F1-Score: 0.8500000000000001  (Harmonic mean of precision and recall, balances false positives and false negatives)
Confusion Matrix:
		Actual Positive	Actual Negative
Predicted Positive	20		2
Predicted Negative	4		17


In [ ]:

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 790, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll